## First Step: Get a session in PGX server.

PGX server requires authentication and TLS connection. By default it uses https://server:7007 as entry point.
This parragraph gets a token valid for 3600 seconds and then creates a client session in PGX.

In [ ]:
import json
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

def generateToken(base_url, username, password):
    from urllib.request import Request, urlopen
    from urllib.error import HTTPError

    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)

    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')

    except HTTPError as err:

        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))

        os._exit(1)

baseUrl = 'https://localhost:7007'
username = 'bankuser'
password = 'welcome1'

token=generateToken(baseUrl, username, password)

Now, with the token, some objects are built:
 * *session* this is the first
 * *instance* created from the session
 * *analyst* this one is used for the algorithms

In [ ]:
try:
    import pypgx as pgx

    session = pgx.get_session(base_url=baseUrl, token=token)
    instance = session.server_instance
    analyst = session.create_analyst()

except SystemExit:
    os._exit(0)
    

Let's print the session ID. This can be used to log in to Graph Viz so that the graphs can be visualized.

In [ ]:
print(session)

Now, all is ready to load the graph from database tables to PGX memory structures.
A json file containing the metadata and mappings from the database table is used for this step.

Let's have a look at the contents of /home/oracle/graphdata/bank_pgviews.json:

```json
{
    "name": "bankpv",
    "vertex_id_strategy": "keys_as_ids",
    "vertex_id_type": "long",
    "vertex_providers": [
        {
            "name": "ACCOUNT",
            "format": "rdbms",
            "database_table_name": "PACCOUNTS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "ACCOUNTNUMBER",
                    "type": "string"
                },
                {
                    "name": "BALANCE",
                    "type": "long"
                }
            ]
        },
        {
            "name": "CUSTOMER",
            "format": "rdbms",
            "database_table_name": "PCUSTOMERS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "FIRSTNAME",
                    "type": "string"
                },
                {
                    "name": "LASTNAME",
                    "type": "string"
                }
            ]
        },
        {
            "name": "CREDITCARD",
            "format": "rdbms",
            "database_table_name": "PCREDITCARDS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "ACCOUNTNUMBER",
                    "type": "string"
                },
                {
                    "name": "LIMIT",
                    "type": "long"
                },
                {
                    "name": "BALANCE",
                    "type": "long"
                },
                {
                    "name": "EXPIRATIONDATE",
                    "type": "timestamp"
                }
            ]
        },
        {
            "name": "PHONENUMBER",
            "format": "rdbms",
            "database_table_name": "PPHONENUMBERS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "PHONENUMBER",
                    "type": "string"
                }
            ]
        },
        {
            "name": "SSN",
            "format": "rdbms",
            "database_table_name": "PSSNS",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "SSN",
                    "type": "string"
                }
            ]
        },
        {
            "name": "ADDRESS",
            "format": "rdbms",
            "database_table_name": "PADDRESSES",
            "key_column": "ID",
            "key_type": "long",
            "props": [
                {
                    "name": "STREET",
                    "type": "string"
                },
                {
                    "name": "CITY",
                    "type": "string"
                },
                {
                    "name": "STATE",
                    "type": "string"
                },
                {
                    "name": "ZIPCODE",
                    "type": "long"
                }
            ]
        }
    ],
    "edge_providers": [
         {
            "name": "HAS_BANKACCOUNT",
            "format": "rdbms",
            "database_table_name": "E_HAS_ACCOUNT",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOACCOUNT",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "ACCOUNT",
            "props": [
               {
                    "name": "CREATIONDATE",
                    "type": "timestamp"
                }
            ]
        },
        {
            "name": "HAS_CREDITCARDS",
            "format": "rdbms",
            "database_table_name": "E_HAS_CREDITCARD",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOCREDITCARD",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "CREDITCARD",
            "props": []
        },
        {
            "name": "HAS_PHONENUMBER",
            "format": "rdbms",
            "database_table_name": "E_HAS_PHONENUMBER",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOPHONE",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "PHONENUMBER",
            "props": []
        },
        {
            "name": "HAS_SSN",
            "format": "rdbms",
            "database_table_name": "E_HAS_SSN",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOSSN",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "SSN",
            "props": []
        },
        {
            "name": "HAS_ADDRESS",
            "format": "rdbms",
            "database_table_name": "E_HAS_ADDRESS",
            "source_column": "FROMCUSTOMER",
            "destination_column": "TOADDRESS",
            "source_vertex_provider": "CUSTOMER",
            "destination_vertex_provider": "ADDRESS",
            "props": []
        }
    ]
}

```

In [ ]:
# read Graph

graph = session.read_graph_with_properties("/home/oracle/graphdata/bank_pgviews.json")


Let's have a look at the number of nodes and edges for this graph.

In [ ]:
print('Edges number: ' + str(graph.num_edges) + ' Nodes number: '+ str(graph.num_vertices))

## Graph visualization with visjs

Let's use [visjs](https://visjs.org/) to visualize the graph data.

To do so, first a PGQL query is executed to get the list of nodes and edges to be represented. Also some of the properties are retrieved to be shown.

In [ ]:
# get result
# node data
pgxResultSetNode = graph.query_pgql("""
  SELECT id(x), label(x), x.FIRSTNAME, x.CITY, x.PHONENUMBER,x.SSN
  MATCH (x)-[]-()
""")

#['size', 'value', 'title', 'x', 'y', 'label', 'color']

node_id = []
node_title =[]
node_label = []

for i in pgxResultSetNode:
    size = i[0]
    if size not in node_id:
        node_id.append(size)
        node_label.append(i[1])
        if i[1] == 'CUSTOMER':
           node_title.append(i[2])
           print('Node: size: ' + str(i[0]) + ', label: ' + i[1] + ', FIRSTNAME: ' + i[2])
        elif i[1] == 'ADDRESS':
           node_title.append(i[3])
           print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' CITY: ' + i[3])
        elif i[1] == 'PHONENUMBER':
           node_title.append(i[4])
           print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' PHONENUMBER: ' + i[4])
        elif i[1] == 'SSN':
           node_title.append(i[5])
           print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' SSN: ' + i[5])
        else:
           node_title.append('n/a') 
           print('Node: size: ' + str(i[0]) + ' label: ' + i[1] + ' title: n/a')
        
        
        
# edge data

pgxResultSetEdge = graph.query_pgql("""
  SELECT id(x), id(y), label(e)
  MATCH (x)-[e]->(y)
""")


edge_list = []
for i in pgxResultSetEdge:
    edge_tuple = (i[0], i[1], i[2])
    edge_list.append(edge_tuple)
    print('Edge: src: ' + str(i[0]) + ' dst: ' + str(i[1]) + ' label: ' + i[2])

Now let's give Visjs a first try.

You can try to enable the controls by uncommenting this line:
```
g.show_buttons()
```

In [ ]:
from pyvis.network import Network

g = Network(notebook=True, height = '800px', width = '100%' , directed = True)


g.add_nodes(node_id,label=node_label,title=node_title)
g.add_edges(edge_list)
#g.show_buttons()
g.show('graph.html')

Let's try nother visualization.
This time let's use some options to tune how nodes are represented.

In [ ]:
from pyvis.network import Network

g2 = Network(notebook=True, height = '800px', width = '100%' ,bgcolor="#f0f0f0", directed = True)
g2.options = {
          "nodes": {
            "scaling": {
              "min": 16,
              "max": 32,
            },
          },
          "edges": {
            "color": "GRAY",
            "smooth": "false",
            "width": "3"
          },
          "physics": {
            "barnesHut": { "gravitationalConstant": -30000 },
            "stabilization": { "iterations": 2500 },
          },
          "groups": {
            "CUSTOMER": {
              "color": { "background": "orange", "border": "black" }
            },
            "PHONENUMBER": {
              "color": "#2B7CE9",
            },
            "ADDRESS": {
              "color": "#5A1E5C",
            },
            "ACCOUNT": {
              "color": "#C5000B",
            },
            "SSN": {
              "color": "#109618",
            }
          }
}

# node data
pgxResultSetNode = graph.query_pgql("""
  SELECT id(x), label(x), x.FIRSTNAME, x.CITY, x.PHONENUMBER,x.SSN
  MATCH (x)-[]-()
""")

for i in pgxResultSetNode:
    if i[1] == 'CUSTOMER':
       g2.add_node(i[0],label=i[2],group=i[1],title=i[2],shape='star')
    elif i[1] == 'ADDRESS':
       g2.add_node(i[0],label=i[3],group=i[1],title=i[3])
    elif i[1] == 'PHONENUMBER':
       g2.add_node(i[0],label=i[4],group=i[1],title=i[4])
    elif i[1] == 'SSN':
       g2.add_node(i[0],label=i[5],group=i[1],title=i[5])
    else:
       g2.add_node(i[0],label=i[1],group=i[1],title='n/a')

        
# edge data

pgxResultSetEdge = graph.query_pgql("""
  SELECT id(x), id(y), label(e)
  MATCH (x)-[e]->(y)
""")

for i in pgxResultSetEdge:
    g2.add_edge(i[0],i[1],title=i[2])
    #print('Edge: src: ' + str(i[0]) + ' dst: ' + str(i[1]) + ' label: ' + i[2])

g2.show('graph.html')

In [ ]:
from pyvis.network import Network
import networkx as nx

nx_graph = nx.Graph(directed = True)

pgxResultSetNode = graph.query_pgql("""
  SELECT id(x), label(x), x.FIRSTNAME, x.CITY, x.PHONENUMBER,x.SSN
  MATCH (x)-[]-()
""")

for i in pgxResultSetNode:
    if i[1] == 'CUSTOMER':
      nx_graph.add_node(i[0], size=30, title=i[2],label=i[2],shape='star',group=i[1])
    elif i[1] == 'ADDRESS':
      nx_graph.add_node(i[0], size=20, label=i[3], shape='diamond' ,group=i[1])
    elif i[1] == 'PHONENUMBER':
      nx_graph.add_node(i[0], size=20, label=i[4], shape='square' ,group=i[1])
    elif i[1] == 'SSN':
      nx_graph.add_node(i[0], size=20, label=i[5], shape='triangle' ,group=i[1])
    else:
      nx_graph.add_node(i[0], size=10, title="" ,group=i[1])
    
# edge data
pgxResultSetEdge = graph.query_pgql("""
  SELECT id(x), id(y), label(e)
  MATCH (x)-[e]->(y)
""")

edge_list = []
for i in pgxResultSetEdge:
    nx_graph.add_edge(i[0], i[1], weight=5)
    
nt = Network(notebook=True, height = '800px', width = '100%')
# populates the nodes and edges data structures
nt.from_nx(nx_graph)


nt.show('nx.html')

When finished, destroy graph and close session to free resources in PGX engine.

In [ ]:
graph.destroy()
session.destroy()